In [7]:
import pickle
import numpy as np
import networkx as nx

N = 1000 # 1000 data points in this training set
k = 10 # Each node in the graph will have edges to its 10 nearest neighbors
S = 50 # need S-nn graph for soft label empirical distribution over S nearest neighbors, S >> k
M = 4 # partition this into 4 parts
num_points = 8 # each example has 8 points
num_features_per_point = 2 # each point has 2 features

folder = '../data/small_toy/'

### Create k-NN Graph, If Necessary

In [8]:
# Load the dataset of points
points = pickle.load(open(folder + 'points.pickle', 'rb'))
assert len(points) == N
for point in points:
    # every point is a 8x2 feature
    assert point.shape[0] == num_points
    assert point.shape[1] == num_features_per_point

# Build the k-nearest neighbor graph from these nodes
knn_graph = nx.Graph()
for i, point in enumerate(points):
    knn_graph.add_node(i, reading=point)

# Load the pairwise distances
distances = pickle.load(open(folder + 'distances.pickle', 'rb'))
assert distances.shape == (N, N)

# Compute the k and S nearest neighbors by sorting the distances
S_nearest_neighbors = {}
for i in range(N):
    distances_for_point = list(zip(range(N), distances[i]))
    distances_for_point.sort(key=lambda x: x[1])

    # Add an edge to each of the closest neighbors, skipping the node itself
    for j in range(1, k + 1):
        if not knn_graph.has_edge(i, distances_for_point[j][0]):
            knn_graph.add_edge(i, distances_for_point[j][0], weight=distances_for_point[j][0])

    # Keep track of the exactly S nearest neighbors - need this for soft labels
    S_nearest_neighbors[i] = [i]
    for j in range(1, S):
        S_nearest_neighbors[i].append(distances_for_point[j][0])

nx.write_gpickle(knn_graph, folder + 'knn_graph.gpickle')
pickle.dump(S_nearest_neighbors, open(folder + 'nearest_neighbors.pickle', 'wb'))

### Partition Graph using KL if Necessary

In [9]:
from networkx.algorithms import community

# import the knn graph
G = nx.read_gpickle(folder + 'knn_graph.gpickle')

# we use a hierarchical partitioning approach to generate m balanced partitions of the graph
# specifically, we choose m to be a power of 2 and repeatedly use Kernighan-Lin to bisect the 
# graph into 2 approximately equal parts
def graph_partition(graph, level):
    if level == 0:
        return [set(graph.nodes())]
    else:
        part1, part2 = community.kernighan_lin_bisection(graph, weight='weight')
        print("done with iteration at level: {}".format(level))
        return graph_partition(graph.subgraph(part1), level - 1) + graph_partition(graph.subgraph(part2), level - 1)

# First, partition the knn graph formed by the training set
num_levels = int(np.log2(M))
assert M == 2 ** num_levels
partition = graph_partition(G, num_levels)

# check the partition was correct
union = []
for part in partition:
    union += list(part)
assert set(union) == set(G.nodes())

pickle.dump(partition, open(folder + "graph_partitions.pickle", "wb"))

done with iteration at level: 2
done with iteration at level: 1
done with iteration at level: 1


### Partition Graph with k-Medioids if Necessary

In [11]:
# import the knn graph
G = nx.read_gpickle(folder + 'knn_graph.gpickle')

from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import DistanceMetric
from tqdm import tqdm_notebook as tqdm
from energyflow.emd import emd, emds
import ot

def ot_distance(x, y):
    x = x.reshape(-1, 2)
    y = y.reshape(-1, 2)
    C = ot.dist(x, y)
    return ot.emd2([], [], C)

points = pickle.load(open(folder + 'points.pickle', 'rb'))

In [18]:
# initialize the medioid indices
medioids = np.random.choice(range(len(points)), size=M, replace=False)
partition = [{} for _ in range(M)]

for i, point in enumerate(points):
    min_distance = ot_distance(point, points[medioids[0]])
    assignment = 0
    for j in range(1, M):
        new_distance = ot_distance(point, points[medioids[j]])
        if new_distance < min_distance:
            assignment = j
            min_distance = new_distance
    partition[j].add(i)

[{}, {}, {}, {}]


### Model Setup

In [148]:
# import the knn graph and partitions
G = nx.read_gpickle(folder + 'knn_graph.gpickle')

# partition_by_component = True
# if partition_by_component:
#     partitions = [set(range(250)), set(range(250, 500)), set(range(500, 750)), set(range(750,1000))]
# else:
#     partitions = nx.read_gpickle(folder + 'graph_partitions.pickle')
graph_partition = nx.read_gpickle(folder + 'graph_partitions.pickle')
km_partitions = nx.read_gpickle(folder + 'km_partitions.pickle')

assert N == len(G.nodes())
assert M == len(graph_partition)
assert M == len(km_partitions)
levels = int(np.log2(M))
assert M == 2 ** levels

# load the training data
points = pickle.load(open(folder + 'points.pickle', 'rb'))
assert len(points) == N

In [10]:
def get_flat_X_unsorted():
    # flatten each training point's feature matrix into a single feature vector
    # note that not every collision event may have the same number of particles
    num_particles = 0
    for point in points:
        num_particles = max(num_particles, point.shape[0])
    num_readings = points[0].shape[1] # every particle should have the same number (3) of readigns

    X = []
    for point in points:
        feature = np.copy(point)
        feature.resize((num_particles * num_readings, ))
        X.append(feature)
    return np.array(X)

def get_flat_X_sorted():
    # flatten each training point's feature matrix into a single feature vector
    # note that not every collision event may have the same number of particles
    num_particles = 0
    for point in points:
        num_particles = max(num_particles, point.shape[0])
    num_readings = points[0].shape[1] # every particle should have the same number (3) of readigns

    X = []
    for point in points:
        feature = np.copy(point)
        
        # sort the point data by the first column
        index = feature[:,0].argsort()
        feature = feature[index]
        
        # now flatten the feature and fill in with 0's
        feature.resize((num_particles * num_readings, ))
        X.append(feature)
    return np.array(X)
    
def get_class_labels(partition):
#     if partition_by_component:
#         labels = []
#         for i in range(250):
#             labels.append(0)
#         for i in range(250, 500):
#             labels.append(1)
#         for i in range(500, 750):
#             labels.append(2)
#         for i in range(750, 1000):
#             labels.append(3)
#         return np.array(labels)
#     else:
    # create labels for each node
    labels_dict = {}
    for i, part in enumerate(partition):
        for node in part:
            labels_dict[node] = i

    labels = []
    for i in range(N):
        labels.append(labels_dict[i])
    return np.array(labels)

def get_hierarchical_labels(partition):
    # return vector of 0 and 1's where each entry determines next splitting point
    labels = get_class_labels(partition)
    
    Y = []
    for label in labels:
        hierarchical_label = []
        for j in range(levels):
            hierarchical_label.append(label // (2 ** (levels - 1 - j)))
            label = label % (2 ** (levels - 1 - j))
        Y.append(hierarchical_label)
    return np.array(Y)

def get_soft_labels(partition):
    # first get regular labels
    labels = get_class_labels(partition)

    # turn the labels into soft labels
    # for this, the label becomes the empirical distribution of the part that each node's S nearest neighbors belong to
    nns = nx.read_gpickle(folder + 'nearest_neighbors.pickle')
    Y = []
    for i in range(N):
        distribution = np.zeros(M)
        for n in nns[i]:
            distribution[labels[n]] += 1
        distribution = np.divide(distribution, np.sum(distribution))
        Y.append(distribution)
    Y = np.array(Y)

    return Y

In [150]:
from sklearn.linear_model import LogisticRegression

def lr_train_hierarchical(models, prefix, X, y, level):
    if level == levels:
        return
    
    clf = LogisticRegression().fit(X, y[:,level])
    models[prefix] = clf
    
    predictions = clf.predict(X)
    
    all_zeros = np.ones(len(predictions))
    X_left = X[predictions == all_zeros]
    y_left = y[predictions == all_zeros]
    lr_train_hierarchical(models, prefix + '0', X_left, y_left, level + 1)
    
    all_ones = np.ones(len(predictions))
    X_right = X[predictions == all_ones]
    y_right = y[predictions == all_ones]
    lr_train_hierarchical(models, prefix + '1', X_right, y_right, level + 1)
    
def lr_predict(model, x, num_levels=levels):
    prediction = []
    prefix = ''
    while len(prefix) < num_levels:
        clf = model[prefix]
        label = clf.predict(x)[0]
        prefix += str(label)
        prediction.append(label)
    return np.array(prediction)
    
def lr_prediction_accuracy(model, X, y):
    num_levels = levels # how deep in the tree to check
    num_correct = 0
    
    # check each data point
    for i in range(len(X)):
        x = X[i:i+1,:]
        prediction = predict(model, x, num_levels)
                
        if (y[i][:num_levels] == prediction).all():
            num_correct += 1
    
    return num_correct / len(X)

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import DistanceMetric
from tqdm import tqdm_notebook as tqdm
from energyflow.emd import emd, emds
import ot

def ot_distance(x, y):
    x = x.reshape(-1, 2)
    y = y.reshape(-1, 2)
    C = ot.dist(x, y)
    return ot.emd2([], [], C)

metric = DistanceMetric.get_metric('pyfunc', func=distance)
nb = NearestNeighbors(metric=distance, algorithm='ball_tree')

### Experiments - KL Partitions

In [ ]:
partition = graph_partition

from sklearn.model_selection import train_test_split

In [151]:
# Sorted Linear Regression
X = get_flat_X_sorted()
y = get_hierarchical_labels(partition)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train the top_level_model
lr_sorted_model = {}
lr_train_hierarchical(lr_sorted_model, "", X_train, y_train, 0)

# now assess test accuracy
print(lr_prediction_accuracy(lr_sorted_model, X_test, y_test))

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [152]:
# Centroid Linear Regression
X = #get_flat_X_sorted()
y = get_hierarchical_labels(partition)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train the top_level_model
lr_centroid_model = {}
lr_train_hierarchical(lr_centroid_model, "", X_train, y_train, 0)

# now assess test accuracy
print(lr_prediction_accuracy(lr_centroid_model, X_test, y_test))

1.0


In [157]:
import time

k = 10
knn_sample = []

t1 = time.time()
for i in range(len(X_test)):
    x = X_test[i:i+1, :]
    label = predict(x, levels)
    assigned_partition = partitions[np.dot(np.power(2, range(levels)[::-1]), label)]
    
    distances = []
    for node_id in assigned_partition:
        d = distance(x, X[node_id:node_id+1, :])
        distances.append((node_id, d))
    distances.sort(key=lambda x: x[1])
    knn_sample.append([u[0] for u in distances[:k]])
t2 = time.time()

print(t2-t1)

4.835469722747803


In [158]:
knn_real = []
t1 = time.time()
for i in range(len(X_test)):
    x = X_test[i:i+1, :]
    distances = []
    for node_id in range(N):
        d = ot_distance(x, X[node_id:node_id+1, :])
        distances.append((node_id, d))
    distances.sort(key=lambda x: x[1])
    knn_real.append([u[0] for u in distances[:k]])
t2 = time.time()

print(t2-t1)

17.400413274765015


In [159]:
# now evaluate accuracy
percents_captured = []
for i in range(len(knn_sample)):
    num_correct = len(np.intersect1d(knn_sample[i], knn_real[i]))
    percents_captured.append(num_correct / k)
print(np.mean(percents_captured))

1.0


### Other Stuff

In [ ]:
import matplotlib.pyplot as plt

labels = get_class_labels(partition)
color_map = []
colors = {0: 'red', 1: 'blue', 2: 'green', 3: 'yellow'}

for i in range(N):
    color_map.append(colors[labels[i]])

nx.draw(G, node_color = color_map)
plt.show()